# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#load the csv file
weather_data = pd.read_csv('output_data/cities.csv')
weather_data

,Unnamed: 0,Country,City,Date,Max Temp,Cloudiness,Humid,Latitude,Longitude,Wind Speed
0,0,US,Bethel,1651936501,50.85,100,78,41.3712,-73.4140,8.99
1,1,MZ,Inhambane,1651936455,73.33,0,76,-23.8650,35.3833,5.86
2,2,US,Jamestown,1651936391,55.02,0,54,42.0970,-79.2353,11.99
3,3,ZA,Port Alfred,1651936515,63.18,79,95,-33.5906,26.8910,5.01
4,4,MX,Salina Cruz,1651936761,86.43,39,57,16.1667,-95.2000,5.44
...,...,...,...,...,...,...,...,...,...,...
580,580,AR,Necochea,1651937059,67.86,100,53,-38.5473,-58.7368,12.84
581,581,IN,Yerraguntla,1651937060,97.11,97,28,14.6372,78.5395,14.52
582,582,RU,Novoagansk,1651937060,18.14,66,86,61.9449,76.6625,9.55
583,583,US,Dawson,1651937061,74.62,0,51,31.7735,-84.4466,14.99


In [5]:
#humidity heat map
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]

# Store Humidity in humidity
humidity = weather_data["Humid"]

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [11]:
#Weather Criteria
city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

city_df

,Unnamed: 0,Country,City,Date,Max Temp,Cloudiness,Humid,Latitude,Longitude,Wind Speed
1,1,MZ,Inhambane,1651936455,73.33,0,76,-23.8650,35.3833,5.86
151,151,ZA,Richards Bay,1651936832,70.70,0,86,-28.7830,32.0377,5.17
158,158,MX,Pueblo Nuevo,1651936835,72.43,0,24,23.3833,-105.3833,2.33
164,164,BR,Rio Grande,1651936838,75.31,0,65,-32.0350,-52.0986,2.93
167,167,US,Saint George,1651936410,77.25,0,19,37.1041,-113.5841,8.05
170,170,FR,Saint-François,1651936669,72.68,0,40,46.4154,3.9054,9.22
201,201,MZ,Chibuto,1651936856,75.02,0,60,-24.6867,33.5306,4.61
270,270,BR,Pedregulho,1651936894,79.43,0,45,-20.2569,-47.4767,1.77
275,275,TR,Denizli Province,1651936899,71.62,0,29,37.8402,29.0698,7.83
299,299,MZ,Manjacaze,1651936911,73.31,0,69,-24.7117,33.8828,4.90


In [12]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,Inhambane,MZ,-23.8650,35.3833,
151,Richards Bay,ZA,-28.7830,32.0377,
158,Pueblo Nuevo,MX,23.3833,-105.3833,
164,Rio Grande,BR,-32.0350,-52.0986,
167,Saint George,US,37.1041,-113.5841,
170,Saint-François,FR,46.4154,3.9054,
201,Chibuto,MZ,-24.6867,33.5306,
270,Pedregulho,BR,-20.2569,-47.4767,
275,Denizli Province,TR,37.8402,29.0698,
299,Manjacaze,MZ,-24.7117,33.8828,


In [24]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [26]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    

    except (KeyError, IndexError):
            print("Missing field/result... skipping.")
        
    print("------------")
    


# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 1: Inhambane.
Closest hotel in Inhambane is Hotel Casa Do Capitão.
------------
Retrieving Results for Index 151: Richards Bay.
Closest hotel in Richards Bay is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 158: Pueblo Nuevo.
Missing field/result... skipping.
------------
Retrieving Results for Index 164: Rio Grande.
Closest hotel in Rio Grande is Hotel Atlântico Rio Grande.
------------
Retrieving Results for Index 167: Saint George.
Closest hotel in Saint George is Hyatt Place St. George / Convention Center.
------------
Retrieving Results for Index 170: Saint-François.
Missing field/result... skipping.
------------
Retrieving Results for Index 201: Chibuto.
Closest hotel in Chibuto is Hotel Chibuto.
------------
Retrieving Results for Index 270: Pedregulho.
Closest hotel in Pedregulho is Hotel Pedregulho.
------------
Retrieving Results for Index 275: Denizli Province.
Closest hotel in Denizli Province is Park Dedeman Deniz

In [27]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,Inhambane,MZ,-23.8650,35.3833,Hotel Casa Do Capitão
151,Richards Bay,ZA,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
158,Pueblo Nuevo,MX,23.3833,-105.3833,
164,Rio Grande,BR,-32.0350,-52.0986,Hotel Atlântico Rio Grande
167,Saint George,US,37.1041,-113.5841,Hyatt Place St. George / Convention Center
170,Saint-François,FR,46.4154,3.9054,
201,Chibuto,MZ,-24.6867,33.5306,Hotel Chibuto
270,Pedregulho,BR,-20.2569,-47.4767,Hotel Pedregulho
275,Denizli Province,TR,37.8402,29.0698,Park Dedeman Denizli Hotel
299,Manjacaze,MZ,-24.7117,33.8828,Hotel Mazino


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [31]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))